In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
import numpy as np

%matplotlib inline

In [3]:
import re

In [4]:
clean_data=gpd.read_file('../data/teis_final_data.csv')

In [5]:
clean_data.head(5)

,Child ID,Notification Date,Fiscal Year,Notification Month,Tenn Region,Fiscal Year.1,third_DOB,Late Referral,Qtr,POE,...,Parent Consent,Initial Eligibility,Initial Eligibility Date,Initial IFSP Date,Latest IFSP,Exit Reason,Exit Date,Referral Source Type Name,Year,geometry
0,104085,2018-01-20,2017-2018,Jan,Middle,2017.0,2010-07-07,1.0,3.0,GN,...,,,,,,Unable to contact,2010-01-25,DCS,2018,None
1,350065,2016-08-17,2016-2017,Aug,Middle,2016.0,2016-10-15,1.0,1.0,UC,...,,,,,,Unable to contact,2016-08-31,PCP,2016,None
2,351755,2016-07-12,2016-2017,Jul,Western,2016.0,2016-11-13,,1.0,NW,...,2016-07-12,Ineligible,8/25/2016,,,Ineligible for Part C,2016-08-25,Parent,2016,None
3,353800,2016-09-14,2016-2017,Sep,Eastern,2016.0,2016-10-01,1.0,1.0,FT,...,,,,,,Referral less than 45 days,2016-09-15,Parent,2016,None
4,353805,2016-08-05,2016-2017,Aug,Middle,2016.0,2016-12-04,,1.0,UC,...,2016-08-15,Eligible,10/5/2016,10/10/2016,,618 - Part B eligibility not determined,2016-12-03,Dept. of Health,2016,None


In [6]:
age = clean_data[['Notification Date','DOB','POE','Child ID','County Name','County SES']]
age

,Notification Date,DOB,POE,Child ID,County Name,County SES
0,2018-01-20,2007-07-07,GN,104085,Davidson,Competitive
1,2016-08-17,2013-10-15,UC,350065,Jackson,At-Risk
2,2016-07-12,2013-11-13,NW,351755,Madison,Transitional
3,2016-09-14,2013-10-01,FT,353800,Hawkins,At-Risk
4,2016-08-05,2013-12-04,UC,353805,Putnam,Transitional
...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,GN,531592,Montgomery,Transitional
115520,2022-07-01,2021-12-07,GN,531593,Davidson,Competitive
115521,2022-07-01,2022-06-21,GN,531594,Montgomery,Transitional
115522,2022-07-01,2022-06-17,GN,531595,Davidson,Competitive


In [7]:
age.dtypes

Notification Date    object
DOB                  object
POE                  object
Child ID             object
County Name          object
County SES           object
dtype: object

In [13]:
age['Notification Date']=pd.to_datetime(age['Notification Date'])

C:\Users\vital\AppData\Local\Temp\ipykernel_6372\710523179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['Notification Date']=pd.to_datetime(age['Notification Date'])


In [14]:
age['DOB']=pd.to_datetime(age['DOB'])

C:\Users\vital\AppData\Local\Temp\ipykernel_6372\356649217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['DOB']=pd.to_datetime(age['DOB'])


In [15]:
age['kids_age'] = (age['Notification Date'] - age['DOB'])


C:\Users\vital\AppData\Local\Temp\ipykernel_6372\1059758760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age['kids_age'] = (age['Notification Date'] - age['DOB'])


In [16]:
age.dtypes

Notification Date     datetime64[ns]
DOB                   datetime64[ns]
POE                           object
Child ID                      object
County Name                   object
County SES                    object
kids_age             timedelta64[ns]
dtype: object

In [17]:
age

,Notification Date,DOB,POE,Child ID,County Name,County SES,kids_age
0,2018-01-20,2007-07-07,GN,104085,Davidson,Competitive,3850 days
1,2016-08-17,2013-10-15,UC,350065,Jackson,At-Risk,1037 days
2,2016-07-12,2013-11-13,NW,351755,Madison,Transitional,972 days
3,2016-09-14,2013-10-01,FT,353800,Hawkins,At-Risk,1079 days
4,2016-08-05,2013-12-04,UC,353805,Putnam,Transitional,975 days
...,...,...,...,...,...,...,...
115519,2022-07-01,2020-05-20,GN,531592,Montgomery,Transitional,772 days
115520,2022-07-01,2021-12-07,GN,531593,Davidson,Competitive,206 days
115521,2022-07-01,2022-06-21,GN,531594,Montgomery,Transitional,10 days
115522,2022-07-01,2022-06-17,GN,531595,Davidson,Competitive,14 days
